In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
!pip install --no-deps seqeval[gpu]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c2d2f4392c63d7fa1dc5c4524515fd71c31ae7286c2a82489ebfc084134ac7c2
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
# Required Library
!pip install seqeval==0.0.12

  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-0.0.12-py3-none-any.whl size=7413 sha256=f215990deb64c61a13e2594f37dc0b2aeea601ead8f2064149252554b83b70c6
  Stored in directory: /root/.cache/pip/wheels/6c/6c/fc/7076d687ba54f32c7be7eaaded97df359ef3c8fee08a2d4efc
Successfully built seqeval
  Attempting uninstall: seqeval
    Found existing installation: seqeval 1.2.2
    Uninstalling seqeval-1.2.2:
      Successfully uninstalled seqeval-1.2.2


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 21.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [ ]:
!pip install pytorch-pretrained-bert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.6 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [ ]:
import numpy as np
import pandas as pd

import spacy
from spacy.training import offsets_to_biluo_tags
nlp = spacy.load("en_core_web_lg")

from tqdm import trange
import torch
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertAdam
from transformers import pipeline
from transformers import BertTokenizer, BertConfig, BertForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification

from seqeval.metrics import classification_report, accuracy_score, f1_score

In [ ]:
prefixes = ('\\n', ) + tuple(nlp.Defaults.prefixes)
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

In [ ]:
#CUSTOM TAGS
entity_dict = {
    'Name': 'NAME',
    'College Name': 'CLG',
    'Degree': 'DEG',
    'Graduation Year': 'GRADYEAR',
    'Years of Experience': 'YOE',
    'Companies worked at': 'COMPANY',
    'Designation': 'DESIG',
    'Skills': 'SKILLS',
    'Location': 'LOC',
    'Email Address': 'EMAIL'
}

In [ ]:
df_resumes = pd.read_json('/content/drive/MyDrive/Dataset/Resumes.json', lines=True)

In [ ]:
df_resumes.head(10)

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN
5,Anvitha Rao\nAutomation developer\n\n- Email m...,"[{'label': ['Skills'], 'points': [{'start': 28...",NaN
6,arjun ks\nSenior Program coordinator - oracle ...,"[{'label': ['Skills'], 'points': [{'start': 34...",NaN
7,"Arun Elumalai\nQA Tester\n\nChennai, Tamil Nad...","[{'label': ['Skills'], 'points': [{'start': 19...",NaN
8,Ashalata Bisoyi\nTransaction Processor - Oracl...,"[{'label': ['Skills'], 'points': [{'start': 17...",NaN
9,Ashok Kunam\nTeam Lead - Microsoft\n\n- Email ...,"[{'label': ['Skills'], 'points': [{'start': 41...",NaN


In [ ]:
#Extras Column has no unique values therefore we drop it

df = df_resumes.drop(['extras'], axis=1)
df.head()

,content,annotation
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20..."


In [ ]:
def mergeIntervals(intervals):
    sorted_by_lower_bound = sorted(intervals, key=lambda tup: tup[0])
    merged = []

    for higher in sorted_by_lower_bound:
        if not merged:
            merged.append(higher)
        else:
            lower = merged[-1]
            if higher[0] <= lower[1]:
                if lower[2] is higher[2]:
                    upper_bound = max(lower[1], higher[1])
                    merged[-1] = (lower[0], upper_bound, lower[2])
                else:
                    if lower[1] > higher[1]:
                        merged[-1] = lower
                    else:
                        merged[-1] = (lower[0], higher[1], higher[2])
            else:
                merged.append(higher)

    return merged

In [ ]:
# From 'annotation' column, we are extracting the starting index, ending index, entity label
# So that we can convert the content in BILOU format

def get_entities(df):

    entities = []

    for i in range(len(df)):
        entity = []

        for annot in df['annotation'][i]:
            try:
                ent = entity_dict[annot['label'][0]]
                start = annot['points'][0]['start']
                end = annot['points'][0]['end'] + 1
                entity.append((start, end, ent))
            except:
                pass

        entity = mergeIntervals(entity)
        entities.append(entity)

    return entities

In [ ]:
# Adding a new column 'entities'
df['entities'] = get_entities(df)
df.head()

,content,annotation,entities
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...","[(0, 12, NAME), (13, 46, DESIG), (49, 58, COMP..."
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...","[(0, 14, NAME), (62, 68, LOC), (104, 148, EMAI..."
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...","[(0, 21, NAME), (22, 31, LOC), (65, 117, EMAIL..."
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...","[(0, 12, NAME), (13, 51, DESIG), (54, 60, COMP..."
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...","[(0, 13, NAME), (14, 22, DESIG), (24, 41, COMP..."


In [ ]:
def get_train_data(df):
    tags = []
    sentences = []

    for i in range(len(df)):
        text = df['content'][i]
        entities = df['entities'][i]

        doc = nlp(text)


        tag = offsets_to_biluo_tags(doc, entities)

        tmp = pd.DataFrame([list(doc), tag]).T
        loc = []
        for i in range(len(tmp)):
            if tmp[0][i].text is '.' and tmp[1][i] is 'O':
                loc.append(i)
        loc.append(len(doc))

        last = 0
        data = []
        for pos in loc:
            data.append([list(doc)[last:pos], tag[last:pos]])
            last = pos

        for d in data:
            tag = ['O' if t is '-' else t for t in d[1]]
            if len(set(tag)) > 1:
                sentences.append(d[0])
                tags.append(tag)

    return sentences, tags

<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-15-9edd3db0ced6>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if tmp[0][i].text is '.' and tmp[1][i] is 'O':
<ipython-input-15-9edd3db0ced6>:17: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if tmp[0][i].text is '.' and tmp[1][i] is 'O':
<ipython-input-15-9edd3db0ced6>:28: SyntaxWarning: "is" with a literal. Did you mean "=="?
  tag = ['O' if t is '-' else t for t in d[1]]


In [ ]:
sentences, tags = get_train_data(df)

len(sentences), len(tags)

/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Afreen Jamadar
Active member of IIIT Committee in ..." with entities "[(0, 14, 'NAME'), (62, 68, 'LOC'), (104, 148, 'EMA...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Akhil Yadav Polemaina
Hyderabad, Telangana - Email..." with entities "[(0, 21, 'NAME'), (22, 31, 'LOC'), (65, 117, 'EMAI...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/spacy/training/iob_utils.py:149: UserWarning: [W030] Some entities could not be 

(780, 780)

In [ ]:
len(sentences), len(tags)
print(sentences[2])
print(tags[2])

[., 
, 
, EDUCATION, 
, 
, B.E, in, Information, science, and, engineering, 
, 
, B.v.b, college, of, engineering, and, technology, -,  , Hubli, ,, Karnataka, 
, 
, August, 2013, to, June, 2017, 
, 
, 12th, in, Mathematics, 
, 
, Woodbine, modern, school, 
, 
, April, 2011, to, March, 2013, 
, 
, 10th, 
, 
, Kendriya, Vidyalaya, 
, 
, April, 2001, to, March, 2011, 
, 
, SKILLS, 
, 
, C, (, Less, than, 1, year, ), ,, Database, (, Less, than, 1, year, ), ,, Database, Management, (, Less, than, 1, year, ), ,, 
, Database, Management, System, (, Less, than, 1, year, ), ,, Java, (, Less, than, 1, year, ), 
, 
, ADDITIONAL, INFORMATION, 
, 
, Technical, Skills, 
, 
, https://www.indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a?isid=rex-download&ikw=download-top&co=IN, 
, 
, 
, •, Programming, language, :, C, ,, C++, ,, Java, 
, •, Oracle, PeopleSoft, 
, •, Internet, Of, Things, 
, •, Machine, Learning, 
, •, Database, Management, System, 
, •, Computer, Networks, 
, •, Operating, System, worked, o

In [ ]:
tag_vals = set(['X', '[CLS]', '[SEP]'])
for i in range(len(tags)):
    tag_vals = tag_vals.union(tags[i])
tag_vals

{'B-CLG',
 'B-COMPANY',
 'B-DEG',
 'B-DESIG',
 'B-EMAIL',
 'B-GRADYEAR',
 'B-LOC',
 'B-NAME',
 'B-SKILLS',
 'B-YOE',
 'I-CLG',
 'I-COMPANY',
 'I-DEG',
 'I-DESIG',
 'I-EMAIL',
 'I-GRADYEAR',
 'I-LOC',
 'I-NAME',
 'I-SKILLS',
 'I-YOE',
 'L-CLG',
 'L-COMPANY',
 'L-DEG',
 'L-DESIG',
 'L-EMAIL',
 'L-GRADYEAR',
 'L-LOC',
 'L-NAME',
 'L-SKILLS',
 'L-YOE',
 'O',
 'U-CLG',
 'U-COMPANY',
 'U-DEG',
 'U-DESIG',
 'U-EMAIL',
 'U-GRADYEAR',
 'U-LOC',
 'U-SKILLS',
 'U-YOE',
 'X',
 '[CLS]',
 '[SEP]'}

In [ ]:
tag2idx = {t: i for i, t in enumerate(tag_vals)}
tag2idx

{'[SEP]': 0,
 'I-GRADYEAR': 1,
 'B-EMAIL': 2,
 'L-NAME': 3,
 'L-EMAIL': 4,
 'B-DEG': 5,
 '[CLS]': 6,
 'U-DEG': 7,
 'O': 8,
 'B-YOE': 9,
 'B-GRADYEAR': 10,
 'I-NAME': 11,
 'U-COMPANY': 12,
 'I-CLG': 13,
 'B-CLG': 14,
 'B-COMPANY': 15,
 'L-YOE': 16,
 'I-DESIG': 17,
 'L-GRADYEAR': 18,
 'L-LOC': 19,
 'I-EMAIL': 20,
 'L-SKILLS': 21,
 'I-COMPANY': 22,
 'L-COMPANY': 23,
 'B-NAME': 24,
 'U-DESIG': 25,
 'I-SKILLS': 26,
 'X': 27,
 'L-DESIG': 28,
 'I-DEG': 29,
 'B-SKILLS': 30,
 'U-GRADYEAR': 31,
 'U-LOC': 32,
 'U-YOE': 33,
 'U-SKILLS': 34,
 'I-LOC': 35,
 'B-DESIG': 36,
 'L-CLG': 37,
 'U-CLG': 38,
 'B-LOC': 39,
 'I-YOE': 40,
 'L-DEG': 41,
 'U-EMAIL': 42}

In [ ]:
idx2tag = {tag2idx[key] : key for key in tag2idx.keys()}
idx2tag

{0: '[SEP]',
 1: 'I-GRADYEAR',
 2: 'B-EMAIL',
 3: 'L-NAME',
 4: 'L-EMAIL',
 5: 'B-DEG',
 6: '[CLS]',
 7: 'U-DEG',
 8: 'O',
 9: 'B-YOE',
 10: 'B-GRADYEAR',
 11: 'I-NAME',
 12: 'U-COMPANY',
 13: 'I-CLG',
 14: 'B-CLG',
 15: 'B-COMPANY',
 16: 'L-YOE',
 17: 'I-DESIG',
 18: 'L-GRADYEAR',
 19: 'L-LOC',
 20: 'I-EMAIL',
 21: 'L-SKILLS',
 22: 'I-COMPANY',
 23: 'L-COMPANY',
 24: 'B-NAME',
 25: 'U-DESIG',
 26: 'I-SKILLS',
 27: 'X',
 28: 'L-DESIG',
 29: 'I-DEG',
 30: 'B-SKILLS',
 31: 'U-GRADYEAR',
 32: 'U-LOC',
 33: 'U-YOE',
 34: 'U-SKILLS',
 35: 'I-LOC',
 36: 'B-DESIG',
 37: 'L-CLG',
 38: 'U-CLG',
 39: 'B-LOC',
 40: 'I-YOE',
 41: 'L-DEG',
 42: 'U-EMAIL'}

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(device)


cuda


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
def get_tokenized_train_data(sentences, tags):

    tokenized_texts = []
    word_piece_labels = []

    for word_list, label in zip(sentences, tags):

        # Add [CLS] at the front
        temp_lable = ['[CLS]']
        temp_token = ['[CLS]']

        for word, lab in zip(word_list, label):
            token_list = tokenizer.tokenize(word.text)
            for m, token in enumerate(token_list):
                temp_token.append(token)
                if m == 0:
                    temp_lable.append(lab)
                else:
                    temp_lable.append('X')

        # Add [SEP] at the end
        temp_lable.append('[SEP]')
        temp_token.append('[SEP]')

        tokenized_texts.append(temp_token)
        word_piece_labels.append(temp_lable)

    return tokenized_texts, word_piece_labels

In [ ]:
tokenized_texts, word_piece_labels = get_tokenized_train_data(sentences, tags)


In [ ]:
print(len(word_piece_labels))
print(tokenized_texts[1])
print(word_piece_labels[1])

780
['[CLS]', '.', 'Will', '##ing', 'to', 'relocate', 'to', ':', 'Bangalore', ',', 'Karnataka', 'W', '##OR', '##K', 'E', '##X', '##P', '##ER', '##IE', '##NC', '##E', 'Application', 'Development', 'Associate', 'A', '##cc', '##ent', '##ure', '-', 'November', '2017', 'to', 'Present', 'Role', ':', 'Currently', 'working', 'on', 'Cha', '##t', '-', 'b', '##ot', '[SEP]']
['[CLS]', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'X', 'O', 'X', 'X', 'X', 'X', 'X', 'X', 'B-DESIG', 'I-DESIG', 'L-DESIG', 'U-COMPANY', 'X', 'X', 'X', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'X', 'O', 'O', 'X', '[SEP]']


In [ ]:
data = {'sentence': sentences,
        'word_labels': word_piece_labels}

test_df = pd.DataFrame(data)

print(test_df.dtypes)

test_df['sentence'] = test_df['sentence'].astype(str)

# Replace the square brackets with nothing
test_df['sentence'] = test_df['sentence'].str.replace('[\[\]]', '', regex=True)



test_df['word_labels'] = test_df['word_labels'].astype(str)

# Replace the square brackets with nothing
test_df['word_labels'] = test_df['word_labels'].str.replace('[\[\]]', '', regex=True)



MAX_LEN = 512
bs = 4
test_df
# df_edited_resumes = df.drop('annotation', axis=1)


# df_edited_resumes

# df_edited_resumes['word_labels'] = word_piece_labels

# df_edited_resumes



sentence       object
word_labels    object
dtype: object


,sentence,word_labels
0,"Abhishek, Jha, \n, Application, Development, A...","'CLS', 'B-NAME', 'X', 'X', 'X', 'X', 'L-NAME',..."
1,"., \n, \n, Willing, to, relocate, to, :, Banga...","'CLS', 'O', 'O', 'X', 'O', 'O', 'O', 'O', 'O',..."
2,"., \n, \n, EDUCATION, \n, \n, B.E, in, Informa...","'CLS', 'O', 'O', 'X', 'X', 'X', 'X', 'B-DESIG'..."
3,"Afreen, Jamadar, \n, Active, member, of, IIIT,...","'CLS', 'B-NAME', 'X', 'X', 'L-NAME', 'X', 'X',..."
4,"., \n, PERSONALLITY, TRAITS, :, \n, •, Quick, ...","'CLS', 'O', 'O', 'X', 'X', 'X', 'X', 'X', 'X',..."
...,...,...
775,"., \n, •, Provisioned, Secure, ID, tokens, ,, ...","'CLS', 'O', 'O', 'O', 'X', 'X', 'O', 'X', 'O',..."
776,"., The, bill, is, for, the, maintenance, \n, o...","'CLS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',..."
777,"., This, approach, was, appreciated, by, the, ...","'CLS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O',..."
778,"Pradeep, Kumar, \n, Security, Analyst, in, Inf...","'CLS', 'B-NAME', 'X', 'X', 'L-NAME', 'B-DESIG'..."


In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))
print(input_ids[0])

512
[  101   138  1830 27516  4638  1377   147  2328 22491  3273  9666   118
   138 19515  3452  3313  7756 12328   117 12247   118 18653 11922  1143
  1113 10364   131  5750   119  3254   120   187   120   138  1830 27516
  4638  1377   118   147  2328   120  1275  1162  1559  1161  1604  1665
  1830  1559 17101  1830  1665 25631  1161   794  1706  1250  1111  1126
  2369  1134  2790  1143  1103  3767  1106  4607  1139  4196  1105  3044
  1111  1139  2510  1105  1419   112   188  3213  1107  1436  1936  3242
   102     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in word_piece_labels], maxlen=MAX_LEN, value=tag2idx["O"],
                     padding="post", dtype="long", truncating="post")
print(len(tags[0]))
print(tags[0])

512
[ 6 24 27 27 27 27  3 27 36 17 28  8 12 27 27 27 32 27  8  8  8  8 27  8
  8  2 20 20 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27 27
 27 27 27 27 27 27 27  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8 27  8  8  8  8  8  0  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8
  8  8  8  8  8  8  8  8  8  8  8  8  8  8  8  

In [ ]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:


tr_inputs, val_inputs, tr_tags, val_tags, tr_masks, val_masks = train_test_split(input_ids, tags, attention_masks, random_state=2020,
                                                                                 test_size=0.3)



In [ ]:


tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)



In [ ]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [ ]:
import torch
from torch import nn
from transformers import BertModel

class BertBiGRU(nn.Module):
    def __init__(self, bert_model, gru_hidden_size, num_layers, num_classes):
        super(BertBiGRU, self).__init__()
        self.bert = bert_model
        self.gru = nn.GRU(bert_model.config.hidden_size, gru_hidden_size, num_layers, bidirectional=True, batch_first=True)
        self.fc = nn.Linear(gru_hidden_size * 2, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs[0]
        gru_out, _ = self.gru(last_hidden_state)
        out = self.fc(gru_out[:, -1, :])
        return out

In [ ]:
bert_model = BertModel.from_pretrained('bert-base-cased')
model = BertBiGRU(bert_model, gru_hidden_size=128, num_layers=2, num_classes=len(tag2idx))

In [ ]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx), id2label =idx2tag, label2id = tag2idx)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.cuda();

In [ ]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [ ]:
epochs = 10
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        outputs = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)

        loss = outputs.loss
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:  10%|█         | 1/10 [00:54<08:06, 54.07s/it]

Train loss: 0.3210541752241824


Epoch:  20%|██        | 2/10 [01:52<07:31, 56.40s/it]

Train loss: 0.09066750697900344


Epoch:  30%|███       | 3/10 [02:49<06:39, 57.06s/it]

...

Train loss: 0.019542329778119812


Epoch:  90%|█████████ | 9/10 [08:37<00:57, 57.92s/it]

Train loss: 0.01576615912911149


Epoch: 100%|██████████| 10/10 [09:35<00:00, 57.60s/it]

Train loss: 0.011129253375663483


In [ ]:
model.eval()

y_true = []
y_pred = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

for batch in valid_dataloader:
    batch = tuple(t.to(device) for t in batch)
    input_ids, input_mask, label_ids = batch

    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None, attention_mask=input_mask,)

        logits = outputs.logits

    logits = logits.detach().cpu().numpy()
    logits = [list(p) for p in np.argmax(logits, axis=2)]

    label_ids = label_ids.to('cpu').numpy()
    input_mask = input_mask.to('cpu').numpy()

    for i,mask in enumerate(input_mask):
        temp_1 = [] # Real one
        temp_2 = [] # Predict one

        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if idx2tag[label_ids[i][j]] != "X" and idx2tag[label_ids[i][j]] != "[CLS]" and idx2tag[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    temp_1.append(idx2tag[label_ids[i][j]])
                    temp_2.append(idx2tag[logits[i][j]])
            else:
                break


        y_true.append(temp_1)
        y_pred.append(temp_2)


print("f1 socre: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

print(classification_report(y_true, y_pred,digits=4))

f1 socre: 0.458927
Accuracy score: 0.829833
           precision    recall  f1-score   support

   SKILLS     0.1376    0.2521    0.1780       119
      LOC     0.6970    0.6161    0.6540       112
    EMAIL     0.5636    0.7209    0.6327        43
  COMPANY     0.5541    0.5503    0.5522       149
 GRADYEAR     0.3922    0.4167    0.4040        48
      CLG     0.2875    0.3151    0.3007        73
    DESIG     0.4367    0.5433    0.4842       127
     NAME     0.8814    0.8814    0.8814        59
      DEG     0.4940    0.5000    0.4970        82
      YOE     0.2222    0.4000    0.2857         5

micro avg     0.4153    0.5129    0.4589       817
macro avg     0.4775    0.5129    0.4907       817



In [ ]:
pipe = pipeline(task="token-classification", model=model.to("cpu"), tokenizer=tokenizer, aggregation_strategy="simple")
pipe("""CAREER ASPIRATIONS
As a second-year university undergraduate and a new entrant to the
corporate working environment, I am looking forward to undertaking an
internship which will enable me to enhance and sharpen my current
theoretical and technical knowledge and skills which will enable me to
perform my duties efficiently and effectively in the organization I intend
serving
TECHNICAL SKILLS
• Programming Languages: Java, Python
• Web Development: HTML5, CSS, JavaScript, PHP
• Database Systems: SQL
• Version Controlling: GitHub, Git
• Development Tools: IntelliJ, Jupyter Notebook, VS Code, PyCharm
PROJECT EXPERIENCE
Clara Vita Smart Adaptive Lamp System 2022
• Led and coordinated a team in the development and
implementation of a smart adaptive lamp system that adapts to
its user’s schedule. Utilized Scikit Learn to develop a machine
learning model that will recognize patterns in the user’s usage
data and identified the algorithm most suited to this project
along with research data to back up the claim.
(This project was awarded 9th place out of 100 teams at
CodeSprint 6.0, 2022)
Van Gogh Archive 2021
• Led and coordinated a team in the development and
implementation of a website that displays paintings and their
respective details about the late artist Vincent Van Gogh, as well
as a mock store page to purchase copies of some of Van Gogh’s
most famous paintings. Personally, developed the souvenir and
presentation pages.
https://w1809849.users.ecs.westminster.ac.uk/presentationPage
.html
SOFT SKILLS DEMONSTRATED:
Excellent Communicator
Strong Leadership Skills
Active Team Player
Conflict Resolution
LANGUAGES:
English – Fluent
Sinhala – Basic Conversational
EXTRA CURRICULAR ACTIVITIES:
Badminton
Swimming
Tennis
INTERESTS:
World History
Technology
Fitness
Computer Games
Investing
ACADEMIC QUALIFICATIONS
Institute of Information Technology, Colombo
(2020 to date)
BEng (Hons) Software Engineering Degree, University of
Westminster, UK
Second Year Module Results:
Database Systems : 91
Object Oriented Programming : 89
Software Development Group Project : Awaiting Results
Server-Side Web Development : Awaiting Results
Machine Learning and Data Mining : Awaiting Results
Algorithms: Theory, Design & Implementation : Awaiting Results
First Year Module Results:
Software Development II : 91
Software Development I : 90
Computer System Fundamentals : 87
Web Design and Development : 86
Mathematics for Computing : 86
Trends in Computer Science : 77""")

[{'entity_group': 'X',
  'score': 0.9997496,
  'word': '##REER',
  'start': None,
  'end': None},
 {'entity_group': 'X',
  'score': 0.999767,
  'word': '##PIRATIONS',
  'start': None,
  'end': None},
 {'entity_group': 'X',
  'score': 0.99923074,
  'word': '##trant',
  'start': None,
  'end': None},
 {'entity_group': 'X',
  'score': 0.9989681,
  'word': '##en',
  'start': None,
  'end': None},
 {'entity_group': 'X',
  'score': 0.9997492,
  'word': '##ECHNICAL',
  'start': None,
  'end': None},
 {'entity_group': 'X',
  'score': 0.99974054,
  'word': '##ILLS',
  'start': None,
  'end': None},
 {'entity_group': 'SKILLS',
  'score': 0.96834856,
  'word': 'Programming Languages : Java , Python • Web Development : H',
  'start': None,
  'end': None},
 {'entity_group': 'X',
  'score': 0.999681,
  'word': '##TML5',
  'start': None,
  'end': None},
 {'entity_group': 'SKILLS',
  'score': 0.9800478,
  'word': ', CS',
  'start': None,
  'end': None},
 {'entity_group': 'X',
  'score': 0.99976987,
  